In [1]:
import gym
import torch
import torch.nn as nn
import torch.functional as F
from torch.distributions.categorical import Categorical

gpu available?

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [4]:
env = gym.make("CartPole-v0")
env.reset()

array([-0.04167364,  0.04348699, -0.03316118,  0.04101069])

In [5]:
batch_size = 5000
epochs = 20

input_size = 4
hidden_size = 32
output_size = 2

Our agent

In [6]:
policy = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size),
    nn.Sigmoid()
).to(device)

In [7]:
env.action_space

Discrete(2)

In [8]:
learning_rate = 1e-2
optimizer = torch.optim.Adam(policy.parameters(), lr=learning_rate)

In [9]:
env.action_space.sample()

1

In [10]:
env.observation_space.sample()

array([-8.9285326e-01,  1.4838560e+38,  1.3352109e-01,  3.9376048e+36],
      dtype=float32)

one training step

In [11]:
def getGrad(policy, states, rew, actions):
    policy.zero_grad()
    expRewrads = [] # [sum(rew)] * len(rew)
    # sum of the following rewards
    for i in range(len(rew)):
        expRewrads.append(sum(rew[i:]))
    _st = torch.as_tensor(states,  dtype=torch.float32, device=device)
    _ac = torch.as_tensor(actions, dtype=torch.int32, device=device)
    out = Categorical(policy(_st)).log_prob(_ac)
    r = torch.as_tensor(expRewrads, dtype=torch.float32, device=device)
    grad = -(out * r).mean()
    return grad

trainig loop

In [12]:
for n in range(epochs):
    obs = env.reset()
    _obs = torch.as_tensor(obs, dtype=torch.float32, device=device)
    trajectories = []
    sa_count = 0
    states  = []
    actions = []
    rewards = []
    while True:
        sa_count += 1
        action = Categorical(policy(_obs)).sample().item()
        states.append(obs.copy())
        actions.append(action)
        obs, reward, done, info = env.step(action)
        _obs = torch.as_tensor(obs, dtype=torch.float32, device=device)
        rewards.append(reward)
        if done:
            trajectories.append((states, rewards, actions))
            states  = []
            actions = []
            rewards = []
            obs = env.reset()
            _obs = torch.as_tensor(obs, dtype=torch.float32, device=device)
            if sa_count > batch_size:
                sa_count = 0
                break

    for trj in trajectories:
        grad = getGrad(policy, trj[0], trj[1], trj[2])
        grad.backward()
        optimizer.step()
    print("avg rewards:", sum(sum(trj[1]) for trj in trajectories)/len(trajectories))
    trajectories = []
print("done")

avg rewards: 22.13274336283186
avg rewards: 24.666666666666668
avg rewards: 26.230366492146597
avg rewards: 30.180722891566266
avg rewards: 40.144
avg rewards: 50.656565656565654
avg rewards: 102.36734693877551
avg rewards: 142.5
avg rewards: 122.4047619047619
avg rewards: 133.1578947368421
avg rewards: 136.32432432432432
avg rewards: 156.625
avg rewards: 162.80645161290323
avg rewards: 169.36666666666667
avg rewards: 150.26470588235293
avg rewards: 181.03571428571428
avg rewards: 186.40740740740742
avg rewards: 189.66666666666666
avg rewards: 194.76923076923077
avg rewards: 199.30769230769232
done


In [13]:
obs = env.reset()
env.render()
obs = torch.as_tensor(obs, dtype=torch.float32, device=device)
print("test")
for _ in range(10):
    rewards = []
    done = False
    while not done:
        env.render()
        # print(obs)
        action = torch.argmax(policy(obs)).item()
        obs, reward, done, info = env.step(action)
        obs = torch.as_tensor(obs, dtype=torch.float32, device=device)
        rewards.append(reward)
        if done:
            print("test rewards", sum(rewards))
            rewards = []
            obs = env.reset()
            obs = torch.as_tensor(obs, dtype=torch.float32, device=device)
env.close()

test
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
test rewards 200.0
